In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get("https://www.shl.com/solutions/products/product-catalog/")
wait = WebDriverWait(driver, 10)

# STEP 1: Collect all assessment links with pagination
assessment_links = set()

while True:
    # wait for the assessment links to load
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/product-catalog/view/']")))
    time.sleep(2)

    # collect current page's assessment links
    for elem in driver.find_elements(By.CSS_SELECTOR, "a[href*='/product-catalog/view/']"):
        link = elem.get_attribute('href')
        if link:
            assessment_links.add(link)

    # find all "Next" buttons on page
    next_buttons = driver.find_elements(By.LINK_TEXT, "Next")

    # if there are at least 2, the second is the Individual Test Solutions pagination
    if len(next_buttons) >= 2:
        btn = next_buttons[1]
    # fallback: if only one Next exists, click it
    elif len(next_buttons) == 1:
        btn = next_buttons[0]
    else:
        print("No more pages to paginate. Finished collecting links.")
        break

    # click and loop
    try:
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(2)
    except Exception:
        print("Failed to click Next. Exiting pagination loop.")
        break

print(f"Total unique assessments found: {len(assessment_links)}")

# STEP 2: Visit each assessment page and extract details
data = []

for link in assessment_links:
    driver.get(link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # -------- Extract Title --------
    try:
        title = driver.find_element(By.TAG_NAME, 'h1').text.strip()
    except:
        title = None

    # -------- Extract Metadata --------
    metadata = {}
    for h4 in driver.find_elements(By.CSS_SELECTOR, "h4"):
        key = h4.text.strip().lower().rstrip(':')
        try:
            val = h4.find_element(By.XPATH, "following-sibling::p[1]").text.strip()
        except:
            val = None
        metadata[key] = val

    description = metadata.get('description')
    if not description:
        try:
            description = driver.find_element(By.CSS_SELECTOR, 'div.description').text.strip()
        except:
            description = None

    job_levels = metadata.get('job levels')
    languages = metadata.get('languages') or metadata.get('report language availability')

    raw_length = metadata.get('assessment length', '') or metadata.get('completion time', '')
    match = re.search(r'(\d+)', raw_length)
    length = match.group(1) if match else None

      # -------- ✅ Extract Test Type (Final, working method) --------
    test_types = []
    try:
        p = driver.find_element(
            By.XPATH,
            "//p[starts-with(normalize-space(text()), 'Test Type:')]"
        )
        raw = p.text  # e.g. "Test Type: A B P"
        test_types = re.findall(r'\b[A-Z]\b', raw)
    except:
        test_types = []
        # -------- Extract Remote Testing (by looking for the green dot) --------
    remote_testing = False
    try:
        # look for the <p> whose text starts with "Remote Testing:"
        for p in driver.find_elements(By.CSS_SELECTOR, "p.product-catalogue__small-text"):
            if p.text.strip().startswith("Remote Testing"):
                # if a green circle is present it will have class "catalogue__circle -yes"
                dots = p.find_elements(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
                remote_testing = len(dots) > 0
                break
    except Exception:
        remote_testing = False


    # -------- Append Extracted Data --------
    data.append({
        'Title': title,
        'Description': description,
        'Job Levels': job_levels,
        'Languages': languages,
        'Assessment Length': length,
        'Test Type': ", ".join(test_types),
        'Remote Testing': remote_testing,
        'URL': link
    })

# STEP 3: Save to CSV
df = pd.DataFrame(data)
df.to_csv('shl_individual_assess.csv', index=False)
print("✅ Saved to shl_individual_assess.csv")
driver.quit() 

No more pages to paginate. Finished collecting links.
Total unique assessments found: 389
✅ Saved to shl_individual_assess.csv


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get("https://www.shl.com/solutions/products/product-catalog/")
wait = WebDriverWait(driver, 10)

# STEP 1: Collect all assessment links with pagination
assessment_links = set()

while True:
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/product-catalog/view/']")))
    time.sleep(2)
    for elem in driver.find_elements(By.CSS_SELECTOR, "a[href*='/product-catalog/view/']"):
        link = elem.get_attribute('href')
        if link:
            assessment_links.add(link)

    try:
        next_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Next")))
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
    except:
        print("No more pages to paginate. Finished collecting links.")
        break

print(f"Total unique assessments found: {len(assessment_links)}")

# STEP 2: Visit each assessment page and extract details
data = []

for link in assessment_links:
    driver.get(link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # -------- Extract Title --------
    try:
        title = driver.find_element(By.TAG_NAME, 'h1').text.strip()
    except:
        title = None

    # -------- Extract Metadata --------
    metadata = {}
    for h4 in driver.find_elements(By.CSS_SELECTOR, "h4"):
        key = h4.text.strip().lower().rstrip(':')
        try:
            val = h4.find_element(By.XPATH, "following-sibling::p[1]").text.strip()
        except:
            val = None
        metadata[key] = val

    description = metadata.get('description')
    if not description:
        try:
            description = driver.find_element(By.CSS_SELECTOR, 'div.description').text.strip()
        except:
            description = None

    job_levels = metadata.get('job levels')
    languages = metadata.get('languages') or metadata.get('report language availability')

    raw_length = metadata.get('assessment length', '') or metadata.get('completion time', '')
    match = re.search(r'(\d+)', raw_length)
    length = match.group(1) if match else None

    # -------- ✅ Extract Test Type (Final, working method) --------
    test_types = []
    try:
        p = driver.find_element(
            By.XPATH,
            "//p[starts-with(normalize-space(text()), 'Test Type:')]"
        )
        raw = p.text  # e.g. "Test Type: A B P"
        test_types = re.findall(r'\b[A-Z]\b', raw)
    except:
        test_types = []

   # -------- Extract Remote Testing (by looking for the green dot) --------
    remote_testing = False
    try:
        # look for the <p> whose text starts with "Remote Testing:"
        for p in driver.find_elements(By.CSS_SELECTOR, "p.product-catalogue__small-text"):
            if p.text.strip().startswith("Remote Testing"):
                # if a green circle is present it will have class "catalogue__circle -yes"
                dots = p.find_elements(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
                remote_testing = len(dots) > 0
                break
    except Exception:
        remote_testing = False


    # --------  Append Extracted Data --------
    data.append({
        'Title': title,
        'Description': description,
        'Job Levels': job_levels,
        'Languages': languages,
        'Assessment Length': length,
        'Test Type': ", ".join(test_types),
        'Remote Testing': remote_testing,
        'URL': link
    })

# STEP 3: Save to CSV
df = pd.DataFrame(data)
df.to_csv('shl_pre_pack.csv', index=False)
print("✅ Saved to shl_pre_pack.csv")
driver.quit()

No more pages to paginate. Finished collecting links.
Total unique assessments found: 153
✅ Saved to shl_pre_pack.csv


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Navigate to the SHL product catalog page
driver.get("https://www.shl.com/products/product-catalog/?start=0&type=1&type=1")
time.sleep(5)  # Wait for the page to load completely

# Initialize a list to store the extracted data
assessments = []

while True:
    print("📄 Scraping current page...")

    # Locate all table rows in the catalog
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        try:
            # Extract all table data cells in the row
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 4:
                continue  # Skip rows that don't have enough columns

            # Extract assessment title and URL
            title_element = cells[0].find_element(By.TAG_NAME, "a")
            title = title_element.text.strip()
            url = title_element.get_attribute("href")

            # Determine if Remote Testing is available
            remote_testing = bool(cells[1].find_elements(By.CLASS_NAME, "catalogue__circle"))

            # Determine if Adaptive/IRT is available
            adaptive_irt = bool(cells[2].find_elements(By.CLASS_NAME, "catalogue__circle"))

            # Extract Test Types
            test_type_elements = cells[3].find_elements(By.CLASS_NAME, "product-catalogue__key")
            test_types = [elem.text.strip() for elem in test_type_elements]

            # Append the extracted information to the list
            assessments.append({
                "Title": title,
                "URL": url,
                "Remote Testing": remote_testing,
                "Adaptive/IRT": adaptive_irt,
                "Test Types": ", ".join(test_types)
            })
        except Exception as e:
            print(f"Error processing row: {e}")
            continue

    # Try to find and click the "Next" button
    try:
        next_buttons = driver.find_elements(By.LINK_TEXT, "Next")
        if not next_buttons:
            print("⛔ No more pages to paginate. Exiting.")
            break

        next_button = next_buttons[-1]  # Use the last Next button
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Wait for the next page to load
    except Exception as e:
        print(f"⚠️ Failed to click Next button: {e}")
        break

# Close the WebDriver
driver.quit()

# Convert the list to a DataFrame and save to CSV
df = pd.DataFrame(assessments)
df.to_csv("shl_pre_pack_table.csv", index=False)
print("✅ Data extraction complete. Saved to 'shl_assessments_with_pagination.csv'.")


📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Setup Selenium WebDriver (same as before)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# ——— Navigate to the Individual Test Solutions listing ———
driver.get("https://www.shl.com/products/product-catalog/?type=1")
time.sleep(5)  # wait for JS to render the table

assessments = []

while True:
    print("📄 Scraping current page...")

    # Grab all rows in the one table on this filtered page
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        try:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 4:
                continue

            # Title & URL
            link = cells[0].find_element(By.TAG_NAME, "a")
            title = link.text.strip()
            url   = link.get_attribute("href")

            # Remote Testing = green dot?
            remote_testing = bool(cells[1].find_elements(By.CLASS_NAME, "catalogue__circle"))

            # Adaptive/IRT = green dot?
            adaptive_irt   = bool(cells[2].find_elements(By.CLASS_NAME, "catalogue__circle"))

            # Test Types
            test_types = [e.text.strip() for e in cells[3].find_elements(By.CLASS_NAME, "product-catalogue__key")]

            assessments.append({
                "Title":         title,
                "URL":           url,
                "Remote Testing":remote_testing,
                "Adaptive/IRT":   adaptive_irt,
                "Test Types":     ", ".join(test_types)
            })
        except Exception as e:
            print(f"Error processing row: {e}")
            continue

    # Pagination: click the last “Next” button if present
    next_buttons = driver.find_elements(By.LINK_TEXT, "Next")
    if not next_buttons:
        print("⛔ No more pages. Exiting loop.")
        break

    try:
        next_buttons[-1].click()
        time.sleep(3)
    except Exception as e:
        print(f"⚠️ Failed to click Next: {e}")
        break

driver.quit()

# Save to CSV (adjusted filename)
df = pd.DataFrame(assessments)
df.to_csv("shl_individual_table.csv", index=False)
print(f"✅ Data extraction complete. {len(df)} rows → shl_individual_table.csv")


📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄 Scraping current page...
📄

In [ ]:
import pandas as pd

# Load your raw CSV
df = pd.read_csv("shl_assessments_raw.csv")

# Rename columns to consistent lowercase, underscore-separated format
df = df.rename(columns={
    "Title": "name",
    "Description": "description",
    "Job Levels": "job_levels",
    "Languages": "languages",
    "Assessment Length": "duration_minutes",
    "TestTypes": "test_type",
    "RemoteTesting": "is_remote",
    "URL": "url",
    "Adaptive/IRT": "is_adaptive"
})

# Strip whitespace and handle missing values
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df["job_levels"] = df["job_levels"].fillna("").str.split(",")
df["languages"] = df["languages"].fillna("").str.split(",")

# Convert types
df["duration_minutes"] = pd.to_numeric(df["duration_minutes"], errors="coerce")
df["is_remote"] = df["is_remote"].astype(bool)
df["is_adaptive"] = df["is_adaptive"].astype(bool)

# Optional: Expand test_type abbreviations
type_map = {
    "A": "Ability",
    "B": "Behavioral",
    "C": "Cognitive",
    "P": "Personality",
    "S": "Skills"
}
def map_test_types(raw):
    if pd.isna(raw): return []
    types = [x.strip() for x in raw.split(",")]
    return [type_map.get(t, t) for t in types]

df["test_type"] = df["test_type"].apply(map_test_types)

# Save cleaned version
df.to_csv("shl_assessments_cleaned.csv", index=False)

print("✅ Cleaned data saved to shl_assessments_cleaned.csv")


In [1]:
import pandas as pd
import ast

# Load your CSV file
df = pd.read_csv('cleaned_dataset.csv')

# List of columns to clean
columns_to_clean = ['job_levels', 'languages', 'test_type']

# Function to remove brackets and join list elements
def clean_column(value):
    try:
        parsed = ast.literal_eval(value)  # safely parse the string as list
        if isinstance(parsed, list):
            return ', '.join(parsed)
        else:
            return value
    except:
        return value  # in case it's already clean or not parsable

# Apply cleaning to each specified column
for col in columns_to_clean:
    df[col] = df[col].apply(clean_column)

# Save cleaned CSV
df.to_csv('cleaned_file.csv', index=False)

print("Cleaning complete! Saved as 'cleaned_file.csv'")


Cleaning complete! Saved as 'cleaned_file.csv'


In [5]:
import json
import pandas as pd

def clean_list(val):
    if isinstance(val, list):
        return ", ".join(val)
    return val

def convert_json_to_csv(json_path, output_csv="metadata_cleaned.csv"):
    # Load the JSON data
    with open(json_path, "r") as f:
        data = json.load(f)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Flatten list fields
    list_columns = ["job_levels", "languages", "main_keywords"]
    for col in list_columns:
        df[col] = df[col].apply(clean_list)

    # Rename main_keywords to keywords
    df.rename(columns={"main_keywords": "keywords"}, inplace=True)

    # Optional: remove unused fields if needed
    columns_to_keep = [
        "id", "embedding_index", "title", "description", "url", "adaptive_support",
        "remote_support", "test_type", "duration_minutes", "job_levels",
        "languages", "keywords"
    ]
    df = df[columns_to_keep]

    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"✅ Converted JSON to cleaned CSV: {output_csv}")
    return df

# Example usage
if __name__ == "__main__":
    convert_json_to_csv("assessment_metadata.json")


✅ Converted JSON to cleaned CSV: metadata_cleaned.csv


In [7]:
import json
import pandas as pd

def clean_list(val):
    if isinstance(val, list):
        return ", ".join(val)
    return val

def convert_json_to_csv(json_path, output_csv="metadata_cleaned.csv"):
    # Load the JSON data
    with open(json_path, "r") as f:
        data = json.load(f)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Flatten list fields
    for col in ["job_levels", "languages", "main_keywords"]:
        df[col] = df[col].apply(clean_list)

    # Rename main_keywords to keywords
    df.rename(columns={"main_keywords": "keywords"}, inplace=True)

    # Select only the required columns
    columns_to_keep = [
        "title", "description", "url", "adaptive_support", "remote_support",
        "test_type", "duration_minutes", "job_levels", "languages", "keywords"
    ]
    df = df[columns_to_keep]

    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"✅ Converted to CSV without id and embedding_index: {output_csv}")
    return df

# Example usage
if __name__ == "__main__":
    convert_json_to_csv("assessment_metadata.json")


✅ Converted to CSV without id and embedding_index: metadata_cleaned.csv


In [1]:
import pandas as pd
import re

def create_combined_text_column(csv_path, output_path="assessments_with_combined_text.csv"):
    df = pd.read_csv(csv_path)

    # Function to clean the title by removing numbers, brackets, and special characters
    def clean_title(text):
        # Remove brackets and their content
        text = re.sub(r'\[.*?\]|\(.*?\)|\{.*?\}', '', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove special characters (keep letters, spaces, and basic punctuation if needed)
        text = re.sub(r'[^\w\s]', '', text)
        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Apply cleaning to title column
    df["cleaned_title"] = df["title"].apply(clean_title)

    def build_combined_text(row):
        title = f"{row['title']} is the title"
        description = row['description']
        adaptive = "Yes" if str(row['adaptive_support']).strip().lower() in ["yes", "true", "1"] else "No"
        remote = "Yes" if str(row['remote_support']).strip().lower() in ["yes", "true", "1"] else "No"
        test_type = row['test_type']
        duration = f"{int(row['duration_minutes'])} min" if pd.notna(row['duration_minutes']) else "unknown duration"
        job_levels = row['job_levels']
        languages = row['languages']
        keywords = row['keywords']

        return (
            f"{title}, {description} "
            f"{adaptive}, have adaptive support and {remote} have remote support. "
            f"{test_type} skills, duration is {duration}, "
            f"it is a {job_levels} in {languages} language, "
            f"{keywords} can be used."
        )

    df["combined_text"] = df.apply(build_combined_text, axis=1)

    df.to_csv(output_path, index=False)
    print(f"✅ Combined text and cleaned title saved to {output_path}")
    return df

# Example run
create_combined_text_column("metadata_cleaned.csv")


✅ Combined text and cleaned title saved to assessments_with_combined_text.csv


,title,description,url,adaptive_support,remote_support,test_type,duration_minutes,job_levels,languages,keywords,cleaned_title,combined_text
0,Contact Center Sales & Service 8.0,the sales service 8.0 jfa is designed for cont...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,"Knowledge & Skills, Personality & Behavior, Ab...",31,entry-level,"english international, english (usa)","service jfa, jfa, jfa designed, sales service,...",Contact Center Sales Service,Contact Center Sales & Service 8.0 is the titl...
1,Contact Center Manager - Short Form,the contact center - manager solution is for m...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,"Personality & Behavior, Personality & Behavior...",50,"manager, supervisor, front line manager",english (usa),"center manager, contact center, center manager...",Contact Center Manager Short Form,Contact Center Manager - Short Form is the tit...
2,Sales Support Specialist Solution,the sales support specialist solution is for e...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,"Personality & Behavior, Personality & Behavior",43,entry-level,english (usa),"sales support, support specialist, specialist ...",Sales Support Specialist Solution,Sales Support Specialist Solution is the title...
3,General Entry Level - All Industries 7.1 Solution,our general entry level all industries 7.1 sol...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,Personality & Behavior,19,entry-level,"german, norwegian, finnish, french (canada), p...","positions industries, level industries, indust...",General Entry Level All Industries Solution,General Entry Level - All Industries 7.1 Solut...
4,Healthcare Aide 7.0 Solution,our healthcare aide 7.0 solution is designed f...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,Personality & Behavior,22,entry-level,"english (usa), french (canada), latin american...","healthcare aide, aide healthcare, health aide,...",Healthcare Aide Solution,"Healthcare Aide 7.0 Solution is the title, our..."
...,...,...,...,...,...,...,...,...,...,...,...,...
513,Android Development (New),multi-choice test that measures the knowledge ...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,Knowledge & Skills,7,"mid-professional, professional individual cont...",english (usa),"android development, android, development new,...",Android Development,"Android Development (New) is the title, multi-..."
514,SAP BW (Business Warehouse) (New),multi-choice test that measures the knowledge ...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,Knowledge & Skills,9,"mid-professional, professional individual cont...",english (usa),"sap bw, sap, sap architecture, bw business, tu...",SAP BW,SAP BW (Business Warehouse) (New) is the title...
515,Core Java (Entry Level) (New),multi-choice test that measures the knowledge ...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,Knowledge & Skills,13,"mid-professional, professional individual cont...",english (usa),"core java, basic java, java, java entry, java ...",Core Java,"Core Java (Entry Level) (New) is the title, mu..."
516,WriteX - Email Writing (Managerial) (New),open response test that evaluates the ability ...,https://www.shl.com/products/product-catalog/v...,Yes,Yes,Knowledge & Skills,15,"entry-level, graduate, manager, mid-profession...",english (usa),"email writing, writex email, grammar email, em...",WriteX Email Writing,WriteX - Email Writing (Managerial) (New) is t...
